# LangChain Basics

Last Update: Jan 10, 2024

In [17]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving requirements.txt to requirements (4).txt
User uploaded file "requirements (4).txt" with length 116 bytes


In [18]:
pip install -r ./requirements (4).txt -q

/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `pip install -r ./requirements (4).txt -q'


Download [requirements.txt](
https://drive.google.com/file/d/1IeTp3JOjhkHYr21tEEh_7X8ozx39v2xc/view?usp=sharing)

In [21]:
# !pip - installs the packages in the base environment
# pip - installs the packages in the virtual environment

In [22]:
pip show langchain

In [23]:
pip show openai

In [25]:
# upgrading langchain
# pip install langchain --upgrade -q

#### Python-dotenv

In [26]:
import os
from dotenv import load_dotenv, find_dotenv

# loading the API Keys (OpenAI, Pinecone) from .env
load_dotenv(find_dotenv(), override=True)

# os.environ.get('PINECONE_API_KEY')

ModuleNotFoundError: No module named 'dotenv'

### ChatModels: GPT-3.5-Turbo and GPT-4

In [ ]:
import warnings
warnings.filterwarnings('ignore', module='langchain')

In [ ]:
from langchain.schema import(
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI

In [ ]:
chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5, max_tokens=1024)
messages = [
    SystemMessage(content='You are a physicist and respond only in German.'),
    HumanMessage(content='explain quantum mechanics in one sentence')
]
output = chat(messages)

In [ ]:
print(output.content)

Quantenmechanik beschreibt das Verhalten von Teilchen auf subatomarer Ebene durch Wahrscheinlichkeiten und Superpositionszustände.


### Prompt Templates

In [ ]:
from langchain import PromptTemplate

In [ ]:
template = '''You are an experienced virologist.
Write a few sentences about the following {virus} in {language}.'''

prompt = PromptTemplate(
    input_variables=['virus', 'language'],
    template=template
)
print(prompt)

input_variables=['language', 'virus'] template='You are an experienced virologist.\nWrite a few sentences about the following {virus} in {language}.'


In [ ]:
from langchain.chat_models import ChatOpenAI
llm = OpenAI(model_name='gpt-3.5-turbo', temperature=0.7)
output = llm(prompt.format(virus='HIV', language='German'))
print(output)

HIV ist ein Virus, das das Immunsystem des menschlichen Körpers angreift und zu AIDS führen kann. HIV wird hauptsächlich durch ungeschützten Geschlechtsverkehr, gemeinsame Nutzung von infizierten Nadeln oder von einer infizierten Mutter auf ihr Kind während der Schwangerschaft, Geburt oder Stillzeit übertragen. Es gibt verschiedene Arten von HIV, aber die häufigste ist HIV-1. Obwohl es keine Heilung für HIV gibt, können antiretrovirale Medikamente das Virus kontrollieren und das Fortschreiten zu AIDS verlangsamen.


### Simple Chains

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5)
template = '''You are an experienced virologist.
Write a few sentences about the following {virus} in {language}.'''

prompt = PromptTemplate(
    input_variables=['virus', 'language'],
    template=template
)

chain = LLMChain(llm=llm, prompt=prompt)
output = chain.run({'virus': 'HSV', 'language': 'french'})


In [ ]:
print(output)

L'HSV, ou le virus de l'herpès simplex, est un virus très répandu qui infecte les humains. Il existe deux types d'HSV : l'HSV-1, qui provoque généralement des éruptions buccales, et l'HSV-2, qui est principalement responsable des infections génitales. Ces virus sont transmis par contact direct avec les lésions cutanées ou les muqueuses infectées. Bien qu'il n'existe pas de traitement curatif pour l'HSV, des médicaments antiviraux peuvent aider à réduire les symptômes et les récidives. Il est important de prendre des précautions pour prévenir la transmission de l'HSV, notamment en évitant les contacts intimes lors d'éruptions actives.


### Sequential Chains

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

llm1 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.7, max_tokens=1024)
prompt1 = PromptTemplate(
    input_variables=['concept'],
    template='''You are an experienced scientist and Python programmer.
    Write a function that implements the concept of {concept}.'''
)
chain1 = LLMChain(llm=llm1, prompt=prompt1)


llm2 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1.2)
prompt2 = PromptTemplate(
    input_variables=['function'],
    template='Given the Python function {function}, describe it as detailed as possible.'
)
chain2 = LLMChain(llm=llm2, prompt=prompt2)

overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)
output = overall_chain.run('softmax')




> Entering new SimpleSequentialChain chain...
Sure! Here's a function that implements the concept of softmax in Python:

```python
import numpy as np

def softmax(x):
    """
    Compute the softmax of a given array of numbers.
    
    Arguments:
    x -- A numpy array of shape (n,) containing the input values.
    
    Returns:
    softmax_vals -- A numpy array of shape (n,) containing the softmax values.
    """
    
    # Ensure the input is a numpy array
    x = np.array(x)
    
    # Compute the exponential of each element in the input array
    exp_vals = np.exp(x)
    
    # Compute the sum of exponential values
    sum_exp = np.sum(exp_vals)
    
    # Compute the softmax values by dividing each exponential value by the sum of exponential values
    softmax_vals = exp_vals / sum_exp
    
    return softmax_vals
```

You can use this function by passing in an array of numbers as input. The function will return an array of the same shape, where each element represents the soft

### LangChain Agents

In [ ]:
pip install langchain_experimental -q

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.chat_models import ChatOpenAI

In [ ]:
llm = ChatOpenAI(temperature=0)
agent_executor = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True
)
agent_executor.run('Calculate the square root of the factorial of 12 and display it with 4 decimal points')

# agent_executor.run('What is the answer to 5.1 ** 7? \
# Generate the Python code for this and run the code.')




> Entering new AgentExecutor chain...
To calculate the square root of the factorial of 12, we need to find the factorial of 12 first and then take the square root of the result. We can use the math module in Python to calculate the square root. To display the result with 4 decimal points, we can use the format function or the round function.

Action: Python_REPL
Action Input: import math
Observation: 
Thought:I have imported the math module, which will allow me to use the square root function.

Action: Python_REPL
Action Input: math.factorial(12)
Observation: 
Thought:I have calculated the factorial of 12, which is 479,001,600.

Action: Python_REPL
Action Input: math.sqrt(479001600)
Observation: 
Thought:I have calculated the square root of the factorial of 12, which is approximately 21,908.9023.

Action: Python_REPL
Action Input: round(math.sqrt(479001600), 4)
Observation: 
Thought:I have rounded the square root of the factorial of 12 to 4 decimal points, which is 21,908.9023.

Fina

'21,908.9023'